In [42]:
import pandas as pd
import datetime
data = pd.read_csv('summer_camp_data.xls.csv', encoding = "cp1251")
display(data.head())

,date,source,medium,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue
0,2020-05-11,google,organic,Не определено,Десктоп,no,no,3120,1233,5240,1048,525,90,86649.0
1,2020-05-11,yandex,cpc,Не определено,Мобайл,yes,no,3302,544,9930,1984,1416,217,244478.0
2,2020-05-11,google,cpc,Не определено,Мобайл,no,no,2970,1450,5460,1090,599,100,105150.0
3,2020-05-11,google,cpc,Не определено,Десктоп,no,no,1875,854,4250,848,407,71,79003.0
4,2020-05-11,yandex,organic,Не определено,Десктоп,no,no,2159,1000,4110,824,351,62,61861.0


In [35]:
data_source  = data.groupby('source').aggregate({'revenue':'sum'})
display(data_source.sort_values(by='revenue', ascending = False).head())

,revenue
source,
yandex,4.674748e+08
google,4.565170e+08
(direct),2.384474e+08
admitad,6.573886e+07
eLama,1.643239e+07


In [40]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52724 entries, 0 to 52723
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                52724 non-null  datetime64[ns]
 1   source              52724 non-null  object        
 2   medium              52724 non-null  object        
 3   delivery_available  52724 non-null  object        
 4   device_type         52724 non-null  object        
 5   promo_activated     52724 non-null  object        
 6   filter_used         52724 non-null  object        
 7   pageviews           52724 non-null  int64         
 8   visits              52724 non-null  int64         
 9   productClick        52724 non-null  int64         
 10  addToCart           52724 non-null  int64         
 11  checkout            52724 non-null  int64         
 12  transactions        52724 non-null  int64         
 13  revenue             52724 non-null  float64   

In [44]:
data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d')

In [45]:
def corona(x):
    cor_date = datetime.date(2020, 5, 20)
    if x > cor_date:
        return 1
    else:
        return 0
data['coronavirus'] = data['date'].apply(corona)

In [49]:
data['mean_revenue'] = data['revenue']/data['transactions']
data_corona  = data.groupby('coronavirus').aggregate({'mean_revenue':'mean'})
display(data_corona)

,mean_revenue
coronavirus,
0,1229.296798
1,1209.443776


In [50]:
data_corona  = data.groupby(['coronavirus', 'promo_activated']).aggregate({'mean_revenue':'mean'})
display(data_corona)

mean_revenue
coronavirus promo_activated              
0           no                1258.130843
            yes               1202.192170
1           no                1169.178819
            yes               1248.165740

In [55]:
data['cr'] = round((100*data['transactions']/data['visits']), 2)
data['day_number'] = data['date'].dt.weekday

In [57]:
data_weekend = data.query('day_number==5 or day_number==6')
data_days = data.query('day_number!=5 and day_number!=6')

In [58]:
from scipy import stats as st
import numpy as np

sample_1 = data_days['cr']
sample_2 = data_weekend['cr']

alpha = 0.05
results = st.ttest_ind(sample_1, sample_2)
print('p-значение:', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу') 

p-значение: 1.0169205948359643e-10
Отвергаем нулевую гипотезу
